In [121]:
import pandas as pd
import numpy as np
from getData import getDataSplits

learning_rate = 10e-6
max_epochs = 100_000
error_threshold = 10e-5
# Set seed for reproducibility
np.random.seed(seed=1234)

In [122]:
train, test = getDataSplits()

print(train.head().to_string(index=False))

x_train = train.copy()
y_train = train['Chance of Admit']
x_train = x_train.drop(columns=['Chance of Admit'])

x_test = test.copy()
y_test = test['Chance of Admit']
x_test = x_test.drop(columns=['Chance of Admit'])

done
 GRE Score  TOEFL Score  University Rating  SOP  LOR  CGPA  Research  Chance of Admit
       316          107                  2  3.5  3.5  8.64         1             0.57
       315          104                  3  4.0  2.5  8.10         0             0.65
       318          107                  3  3.0  3.5  8.27         1             0.74
       314          105                  2  2.5  2.0  7.64         0             0.70
       301          107                  3  3.5  3.5  8.34         1             0.62


### Let's recap things

Model is Linear Regression:

y_hat = w.T * x + b

We will need to set an loss function to minimize. We will use the Mean Squared Error (MSE) function.

MSE = 1/n * sum((y_hat - y)^2) =>
L = 1/n * sum((w.T * x + b - y)^2)

We will use the gradient descent algorithm to minimize the loss function. The gradient of the loss function with respect to the weights and bias is given by:

Repeat until convergence
w <- w - alpha * dL/dw
b <- b - alpha * dL/db

Where alpha is the learning rate.
We'll add b to w matrix. So, w = [w, b]
So now we need to minimize ||w.T * x - y||^2

The gradient of the loss function with respect to the weights and bias is given by:
dL/dw = 2/n * sum((w.T * x - y) * x)

We need to chose a learning rate and a Error to run the algorithm.

alpha = 10E-2 or 10E-3
Error = 10E-3 or 10E-4

iterations = 0

In [123]:
x_train = x_train.to_numpy()
y_train = y_train.to_numpy()

x_test = x_test.to_numpy()
y_test = y_test.to_numpy()

In [124]:
y_train.shape

learning_rate/len(x_train)

2.5000000000000002e-08

In [125]:
x_train.shape
# add the bias term
w = np.array([1])
# add the weights for each feature randomly
w = np.append(w, np.random.rand(x_train.shape[1]))


delta_Error = 1
old_Error = 0
ephocs = 0

In [126]:
def MSE(y, y_hat):
    return np.mean((y - y_hat) ** 2)

In [127]:
while delta_Error > error_threshold:

    # if we reach the max number of epochs, we stop
    ephocs += 1
    if ephocs > max_epochs:
        break

    y_hat = np.dot(x_train, w[1:]) + w[0]
    print(w, "\n")
    print(y_hat[:5], "\n")
    print(y_train[:5], "\n")

   
    
    w[0] = w[0] - learning_rate/len(x_train) * np.sum(y_hat - y_train)

    w[1:] = w[1:] - learning_rate/len(x_train) * np.dot((y_hat - y_train), x_train)

    error = MSE(y_train, y_hat) 
    print(error, "\n")

    if ephocs == 2:
        break

    delta_Error = abs(old_Error - error)
    old_Error = error

    if delta_Error < error_threshold:
        print('we reached the error threshold')
        print(f'Error: {error} Delta Error: {delta_Error} Epochs: {ephocs}')



[1.         0.444798   0.02211118 0.60598834 0.96479652 0.96759148
 0.93002881 0.18467983] 

[160.11752686 159.04029506 160.78660227 155.65281612 153.77253659] 

[0.57 0.65 0.74 0.7  0.62] 

25610.040097322828 

[ 0.99840136 -0.06155709 -0.14956502  0.6009432   0.95939457  0.96196694
  0.91630951  0.18377159] 

[-18.4297593  -18.47941232 -18.77066209 -21.50994116 -17.18035259] 

[0.57 0.65 0.74 0.7  0.62] 

357.93733359035707 

